In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

In [3]:
load_dotenv()

True

In [4]:
google_api_key = os.getenv("GOOGLE_API")

In [6]:

llm = ChatGoogleGenerativeAI(model="gemini-pro")

message = [HumanMessage(content='hay dude,whats going on')]

responce = llm.invoke(message)

print(responce.content)

**Response 1**

Yo, what's up, dude? What's good with you?

**Response 2**

Hey there, bud! Nothing much, just chillin'. How's it hangin' with you?

**Response 3**

Sup, man? How's your day been so far? Anything exciting happening?

**Response 4**

Yo, my dude! What's crackin'? You up to anything cool today?

**Response 5**

Hey, bro! Just cruising along. What's on your mind?
